Create a mysql database

In [19]:
%pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import mysql.connector

# Step 1: Connect without specifying the database
conn = mysql.connector.connect(host="localhost", user="root", password="")
my_cursor = conn.cursor()

# Step 2: Create the database if it doesn’t exist
my_cursor.execute("CREATE DATABASE IF NOT EXISTS GLOBAL_ELECTRONICS_DETAILS")
my_cursor.close()
conn.close()

# Step 3: Reconnect with the new database
conn = mysql.connector.connect(host="localhost", user="root", password="", database="GLOBAL_ELECTRONICS_DETAILS")
my_cursor = conn.cursor()
print("Database connection successful.")


Database connection successful.


In [1]:
import mysql.connector
import pandas as pd

# Step 1: Load the data from the CSV file
final_data = pd.read_csv(r'C:\Soundhar\Project\DataSpark Illuminating Insights for Global Electronics\Cleaned data\final_data.csv')

In [2]:
final_data.head()

,order_number,line_item,order_date,delivery_date,customerkey,storekey,productkey,quantity,currency_code,processing_time_days,...,categorykey,category,profit_margin,store_key,country_y,state_y,square_meters,open_date,currency,exchange_rate
0,366000,1,01-01-2016,01-01-1900,265598,10,1304,1,CAD,NaN,...,4,Cameras and camcorders,36.73,10,Canada,Nunavut,1210,04-04-2015,CAD,1.3884
1,366001,1,01-01-2016,13-01-2016,1269051,0,1048,2,USD,12.0,...,4,Cameras and camcorders,285.53,0,Online,Online,0,01-01-2010,USD,1.0000
2,366001,2,01-01-2016,13-01-2016,1269051,0,2007,1,USD,12.0,...,8,Home Appliances,445.30,0,Online,Online,0,01-01-2010,USD,1.0000
3,366002,1,01-01-2016,12-01-2016,266019,0,1106,7,CAD,11.0,...,4,Cameras and camcorders,173.92,0,Online,Online,0,01-01-2010,CAD,1.3884
4,366002,2,01-01-2016,12-01-2016,266019,0,373,1,CAD,11.0,...,3,Computers,159.80,0,Online,Online,0,01-01-2010,CAD,1.3884


In [3]:

final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   order_number          62884 non-null  int64  
 1   line_item             62884 non-null  int64  
 2   order_date            62884 non-null  object 
 3   delivery_date         62884 non-null  object 
 4   customerkey           62884 non-null  int64  
 5   storekey              62884 non-null  int64  
 6   productkey            62884 non-null  int64  
 7   quantity              62884 non-null  int64  
 8   currency_code         62884 non-null  object 
 9   processing_time_days  13165 non-null  float64
 10  gender                62884 non-null  object 
 11  name                  62884 non-null  object 
 12  city                  62884 non-null  object 
 13  state_code            62884 non-null  object 
 14  state_x               62884 non-null  object 
 15  zip_code           

In [4]:
import pandas as pd
import mysql.connector

# Load the dataset
file_path = r'C:\Soundhar\Project\DataSpark Illuminating Insights for Global Electronics\Cleaned data\final_data.csv'
df = pd.read_csv(file_path)
print(f"Loaded DataFrame with {df.shape[0]} rows and {df.shape[1]} columns")

# Replace NaN or NaT values with None for SQL compatibility
df = df.replace({pd.NA: None, pd.NaT: None, float('nan'): None})

# Define columns
columns = [
    'order_number', 'line_item', 'order_date', 'delivery_date', 'customerkey', 'storekey',
    'productkey', 'quantity', 'currency_code', 'processing_time_days', 'gender', 'name', 'city',
    'state_code', 'state_x', 'zip_code', 'country_x', 'continent', 'birthday', 'age', 'product_name',
    'brand', 'color', 'unit_cost_usd', 'unit_price_usd', 'subcategorykey', 'subcategory', 'categorykey',
    'category', 'profit_margin', 'store_key', 'country_y', 'state_y', 'square_meters', 'open_date',
    'currency', 'exchange_rate'
]

# Ensure only the required columns are used
df = df[columns]

try:
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="global_electronics_details",
        autocommit=True
    )
    cursor = connection.cursor()
    print("Connected to the database successfully!")

    # Create table if not exists
    create_table_query = """
    CREATE TABLE IF NOT EXISTS finaldata (
        order_number INT, line_item INT, order_date DATE, delivery_date DATE,
        customerkey INT, storekey INT, productkey INT, quantity INT, currency_code VARCHAR(10),
        processing_time_days FLOAT, gender VARCHAR(10), name VARCHAR(100), city VARCHAR(100),
        state_code VARCHAR(10), state_x VARCHAR(100), zip_code VARCHAR(20), country_x VARCHAR(100),
        continent VARCHAR(50), birthday DATE, age INT, product_name VARCHAR(200), brand VARCHAR(100),
        color VARCHAR(50), unit_cost_usd FLOAT, unit_price_usd FLOAT, subcategorykey INT,
        subcategory VARCHAR(100), categorykey INT, category VARCHAR(100), profit_margin FLOAT,
        store_key INT, country_y VARCHAR(100), state_y VARCHAR(100), square_meters INT, open_date DATE,
        currency VARCHAR(10), exchange_rate FLOAT
    )
    """
    cursor.execute(create_table_query)
    print("Table 'finaldata' created or already exists.")

    # Prepare insert query
    insert_query = f"INSERT INTO finaldata ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"

    # Convert the DataFrame to tuples
    data_tuples = [tuple(row) for row in df.itertuples(index=False, name=None)]

    # Insert data in batches
    batch_size = 1000
    for start in range(0, len(data_tuples), batch_size):
        batch = data_tuples[start:start + batch_size]
        try:
            cursor.executemany(insert_query, batch)
            connection.commit()
            print(f"Inserted {len(batch)} rows from index {start} to {start + len(batch) - 1}")
        except mysql.connector.Error as batch_err:
            print(f"Error inserting batch at index {start}: {batch_err}")

except mysql.connector.Error as err:
    print(f"Database error: {err}")
finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("Database connection closed.")


Loaded DataFrame with 62884 rows and 37 columns
Connected to the database successfully!
Table 'finaldata' created or already exists.
Inserted 1000 rows from index 0 to 999
Inserted 1000 rows from index 1000 to 1999
Inserted 1000 rows from index 2000 to 2999
Inserted 1000 rows from index 3000 to 3999
Inserted 1000 rows from index 4000 to 4999
Inserted 1000 rows from index 5000 to 5999
Inserted 1000 rows from index 6000 to 6999
Inserted 1000 rows from index 7000 to 7999
Inserted 1000 rows from index 8000 to 8999
Inserted 1000 rows from index 9000 to 9999
Inserted 1000 rows from index 10000 to 10999
Inserted 1000 rows from index 11000 to 11999
Inserted 1000 rows from index 12000 to 12999
Inserted 1000 rows from index 13000 to 13999
Inserted 1000 rows from index 14000 to 14999
Inserted 1000 rows from index 15000 to 15999
Inserted 1000 rows from index 16000 to 16999
Inserted 1000 rows from index 17000 to 17999
Inserted 1000 rows from index 18000 to 18999
Inserted 1000 rows from index 19000 